In [17]:
import pandas as pd
import os
import mysql.connector
import re
# Folder path containing the CSV files
folder_path = r"D:\GUVI\IMDB2024\Data Scrapping"

#  list of all CSV files in the folder 
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".csv")][:6]

# Read and merge the data
df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# Remove duplicates based on "Title" column (keep the first occurrence)
df.drop_duplicates(subset=["Title"], keep="first", inplace=True)
df["voting"] = df["voting"].astype(str)

# Remove commas
df["voting"] = df["voting"].str.replace(",", "", regex=False)

# Convert K / M if present
df["voting"] = df["voting"].str.replace("K", "000", regex=False)
df["voting"] = df["voting"].str.replace("M", "000000", regex=False)

# Convert to numeric safely
df["voting"] = pd.to_numeric(df["voting"], errors="coerce")

# Replace NaN with 0 (optional but safe)
df["voting"] = df["voting"].fillna(0)
#  Force integer
df["voting"] = df["voting"].astype(int)


def convert_duration(duration):
    if pd.isna(duration):
        return 0

    duration = str(duration).strip()

    # Case 1: "1h 42m"
    match = re.match(r"(\d+)h\s*(\d*)m?", duration)
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2)) if match.group(2) else 0
        return hours * 60 + minutes

    # Case 2: "75m"
    match = re.match(r"(\d+)m", duration)
    if match:
        return int(match.group(1))

    # Case 3: pure number like "90"
    if duration.isdigit():
        return int(duration)

    return 0
df["Duration"] = df["Duration"].apply(convert_duration)
# Remove rows where important fields are missing
df = df.dropna(subset=["Title", "Genre", "Rating", "voting"])

# Remove invalid duration rows
df = df[df["Duration"] > 0]

#  Save the cleaned merged data 
output_file = os.path.join(folder_path, "merged_cleaned_movies.csv")
df.to_csv(output_file, index=False)

print(f"Merged CSV file saved as: {output_file}")

Merged CSV file saved as: D:\GUVI\IMDB2024\Data Scrapping\merged_cleaned_movies.csv


In [19]:
import pandas as pd
import numpy as np
import mysql.connector
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Mysql@123",   # the password you set
    database="movies_database"
)

In [20]:
mycursor = connection.cursor(buffered=True)

# Read CSV
df = pd.read_csv(r"D:\GUVI\IMDB2024\Data Scrapping\merged_cleaned_movies.csv")

# Replace NaN with NULL
df = df.replace({np.nan: None})

# Insert data
for _, row in df.iterrows():
    mycursor.execute("INSERT INTO movies (Title, Genre, Duration, Rating, voting) VALUES (%s, %s, %s, %s, %s)",
        (row["Title"], row["Genre"], row["Duration"], row["Rating"], row["voting"]))
    #print(mycursor.fetchone())
connection.commit()
mycursor.close()
connection.close()

print("Data uploaded successfully!")

Data uploaded successfully!
